In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sksurv.nonparametric import kaplan_meier_estimator

import warnings

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

## Análisis de Sobrevivencia

En este cuadernillo se quiere realizar el análisis de sobrevivencia de pacientes con
trasplante cardiaco en el Instituto Nacional del Tórax. Para este efecto, se utilizará
la base de datos de GRD de pacientes que se hayan realizado un trasplante cardiaco. La base
sigue a pacientes entre 2011 hasta el 16/08/2023 del año 2022.

Las respuestas que se quieren seguir son las siguientes:

- Luego de 2 años del trasplante, ¿Cuál es la probabilidad de que el paciente siga vivo?
- Luego de 10 años del trasplante, ¿Cuál es la probabilidad de que el paciente siga vivo?

Además, se quiere saber cómo es la curva de sobrevivencia para distintos rangos etarios de
pacientes.

In [2]:
df = pd.read_excel("../data/raw/Trasplantes 2011-2022 16082023.xlsx", sheet_name="bdgrd")
df["rango_etario"] = pd.cut(df["Edad en años"], bins=range(0, 121, 10))

In [3]:
columnas_utiles = ["1ra Intervención Fecha", "Fecha fallecimiento", "rango_etario"]
fecha_corte = pd.to_datetime("16/08/2023", dayfirst=True)

In [4]:
df_sobrevivencia = df[columnas_utiles].copy()
df_sobrevivencia.columns = ["fecha_trasplante", "fecha_muerte", "rango_etario"]

In [5]:
df_sobrevivencia["fecha_tope"] = df_sobrevivencia["fecha_muerte"].fillna(fecha_corte)
df_sobrevivencia["duracion_anios"] = (
    df_sobrevivencia["fecha_tope"] - df_sobrevivencia["fecha_trasplante"]
).dt.days / 365
df_sobrevivencia["muerto"] = df_sobrevivencia["fecha_muerte"].notna()

In [6]:
time, survival_prob, conf_init = kaplan_meier_estimator(
    df_sobrevivencia["muerto"], df_sobrevivencia["duracion_anios"], conf_type="log-log"
)

In [7]:
# plt.step(time, survival_prob, where="post")
# plt.fill_between(time, conf_init[0], conf_init[1], alpha=0.25, step="post")
# plt.ylim(0, 1)
# plt.ylabel("est. probability of survival $\hat{S}(t)$")
# plt.xlabel("time $t$")

In [8]:
for rango_etario in df_sobrevivencia["rango_etario"].unique():
    df_rango_etario = df_sobrevivencia.query("rango_etario == @rango_etario")
    time, survival_prob, conf_int = kaplan_meier_estimator(df_rango_etario["muerto"], 
                                                           df_rango_etario["duracion_anios"], 
                                                           conf_type="log-log")
    
#     plt.step(time, survival_prob, where="post", label=f"{rango_etario} n = {df_rango_etario.shape[0]}")
#     # plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")

# plt.ylim(0, 1)
# plt.ylabel("est. probability of survival $\hat{S}(t)$")
# plt.xlabel("time $t$")
# plt.legend(loc="best")